In [1]:
!pip install country_converter

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import country_converter as coco

import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
from PIL import Image
import requests

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
books = pd.read_csv("./books.csv")
ratings = pd.read_csv("./ratings.csv")
users = pd.read_csv("./users.csv")

In [3]:
books.columns

Index(['Branch', 'Book Title', 'Year of Publication', 'Author', 'Semester',
       'Price'],
      dtype='object')

In [4]:
ratings.columns

Index(['UserID', 'Semester', 'Ratings'], dtype='object')

In [5]:
users.columns

Index(['UserID', 'Semester'], dtype='object')

In [6]:
books.isnull().sum()

Branch                 0
Book Title             0
Year of Publication    0
Author                 0
Semester               0
Price                  0
dtype: int64

In [7]:
ratings.isnull().sum()

UserID      0
Semester    0
Ratings     0
dtype: int64

In [8]:
users.isnull().sum()

UserID      0
Semester    0
dtype: int64

### creating database and tables

In [9]:
import mysql.connector
from mysql.connector import Error
# Database connection details
HOST = "localhost"
USER = "root"
PASSWORD = "root"
try:
    # Connect to MySQL server
    connection = mysql.connector.connect(
        host=HOST,
        user=USER,
        password=PASSWORD
    )
    if connection.is_connected():
        print(":white_check_mark: Connected to MySQL Server")
        # Create a cursor object
        cursor = connection.cursor()
        # Step 1: Create the database
        cursor.execute("CREATE DATABASE IF NOT EXISTS educationbr")
        print(":white_check_mark: Database 'educationbookrecommendation' created successfully")
        # Switch to the new database
        cursor.execute("USE educationbr")
        # Step 2: Create Books table
        create_books_table = """
        CREATE TABLE IF NOT EXISTS Books (
            Branch VARCHAR(100) NOT NULL,
            Book_Title VARCHAR(255) NOT NULL,
            Year_Of_Publication INT NOT NULL,
            Author VARCHAR(255) NOT NULL,
            Semester INT NOT NULL,
            Price DECIMAL(10,2)
        )
        """
        cursor.execute(create_books_table)
        print(":white_check_mark: Table 'Books' created successfully")
        # Step 3: Create Users table
        create_users_table = """
        CREATE TABLE IF NOT EXISTS Users (
            User_ID INT PRIMARY KEY,
             Semester INT NOT NULL
        )
        """
        cursor.execute(create_users_table)
        print(":white_check_mark: Table 'Users' created successfully")
        # Step 4: Create Ratings table
        create_ratings_table = """
        CREATE TABLE IF NOT EXISTS Ratings (
            User_ID INT NOT NULL,
            Semester INT NOT NULL,
            Ratings INT CHECK (Ratings BETWEEN 1 AND 10),
            PRIMARY KEY (User_ID, Semester),
            FOREIGN KEY (User_ID) REFERENCES Users(User_ID) ON DELETE CASCADE
        )
        """
        cursor.execute(create_ratings_table)
        print(":white_check_mark: Table 'Ratings' created successfully")
except Error as e:
    print(f":x: Error: {e}")
finally:
    # Close the connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print(":white_check_mark: MySQL connection is closed")


:white_check_mark: Connected to MySQL Server
:white_check_mark: Database 'educationbookrecommendation' created successfully
:white_check_mark: Table 'Books' created successfully
:white_check_mark: Table 'Users' created successfully
:white_check_mark: Table 'Ratings' created successfully
:white_check_mark: MySQL connection is closed


### inserting the data into database

In [10]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
# Database connection details
HOST = "localhost"
USER = "root"
PASSWORD = "root"
DATABASE = "educationbr"
# File paths
BOOKS_CSV = "./books.csv"
USERS_CSV = "./users.csv"
RATINGS_CSV = "./ratings.csv"
try:
    # Connect to MySQL
    connection = mysql.connector.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DATABASE
    )
    if connection.is_connected():
        print(":white_check_mark: Connected to MySQL Database")
        cursor = connection.cursor()
        ### :small_blue_diamond: Insert data into Users table ###
        users_df = pd.read_csv(USERS_CSV)
        users_df.rename(columns={"UserID": "User_ID"}, inplace=True)
        for _, row in users_df.iterrows():
            try:
                cursor.execute("""
                    INSERT IGNORE INTO Users (User_ID, Semester)
                    VALUES (%s, %s)
                """, (int(row["User_ID"]), int(row["Semester"])))
            except Exception as e:
                print(f":x: Error inserting user {row['User_ID']}: {e}")
        connection.commit()  # :white_check_mark: Commit after Users table
        print(":white_check_mark: Users data inserted successfully")
        ### :small_blue_diamond: Insert data into Books table ###
        books_df = pd.read_csv(BOOKS_CSV)
        books_df.rename(columns={
            "Book Title": "Book_Title",
            "Year of Publication": "Year_Of_Publication"
        }, inplace=True)
        for _, row in books_df.iterrows():
            try:
                cursor.execute("""
                    INSERT IGNORE INTO Books (Branch, Book_Title, Year_Of_Publication, Author, Semester, Price)
                    VALUES (%s, %s, %s, %s, %s, %s)
                """, (row["Branch"], row["Book_Title"], int(row["Year_Of_Publication"]),
                      row["Author"], int(row["Semester"]),
                      float(row["Price"]) if pd.notna(row["Price"]) else None))
            except Exception as e:
                print(f":x: Error inserting book {row['Book_Title']}: {e}")
        connection.commit()  # :white_check_mark: Commit after Books table
        print(":white_check_mark: Books data inserted successfully")
        ### :small_blue_diamond: Insert data into Ratings table ###
        ratings_df = pd.read_csv(RATINGS_CSV)
        ratings_df.rename(columns={"UserID": "User_ID"}, inplace=True)
        for _, row in ratings_df.iterrows():
            try:
                cursor.execute("""
                    INSERT IGNORE INTO Ratings (User_ID, Semester, Ratings)
                    VALUES (%s, %s, %s)
                """, (int(row["User_ID"]), int(row["Semester"]),
                      int(row["Ratings"]) if pd.notna(row["Ratings"]) else None))
            except mysql.connector.IntegrityError as e:
                print(f":warning: Duplicate entry skipped for User_ID: {row['User_ID']} in Semester: {row['Semester']}")
            except Exception as e:
                print(f":x: Error inserting rating {row['User_ID']}: {e}")
        connection.commit()  # :white_check_mark: Commit after Ratings table
        print(":white_check_mark: Ratings data inserted successfully")
except Error as e:
    print(f":x: MySQL Error: {e}")
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print(":white_check_mark: MySQL connection is closed")

:white_check_mark: Connected to MySQL Database
:white_check_mark: Users data inserted successfully
:white_check_mark: Books data inserted successfully
:white_check_mark: Ratings data inserted successfully
:white_check_mark: MySQL connection is closed


### extracting the data from database to python

In [11]:
##extracting
import mysql.connector
import pandas as pd
# :white_check_mark: Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="educationbr"
)
cursor = conn.cursor()
# :white_check_mark: Define function to fetch data from a table
def fetch_table_data(table_name):
    query = f"SELECT * FROM {table_name};"
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    return pd.DataFrame(rows, columns=columns)
# :white_check_mark: Fetch data from each table
users_df = fetch_table_data("users")
books_df = fetch_table_data("books")
ratings_df = fetch_table_data("ratings")
# :white_check_mark: Close the connection
cursor.close()
conn.close()
# :white_check_mark: Display the first 5 rows of each table
print("Users Table:")
display(users_df.head(), "\n")
print("Books Table:")
display(books_df.head(), "\n")
print("Ratings Table:")
display(ratings_df.head(), "\n")


Users Table:


,User_ID,Semester
0,1,3
1,2,6
2,3,1
3,4,1
4,5,1


'\n'

Books Table:


,Branch,Book_Title,Year_Of_Publication,Author,Semester,Price
0,Chemical Engineering,Heat Transfer - A Comprehensive Guide,2010,Stuart Russell,6,2127.00
1,Computer Science Engineering,Computer Networks - A Comprehensive Guide,2023,Robert L. Norton,6,1611.00
2,Computer Science Engineering,Operating Systems - A Comprehensive Guide,2006,K.S. Mano,6,1826.00
3,Electronics and Communication Engineering,Digital Electronics - A Comprehensive Guide,2010,Robert L. Norton,4,1002.00
4,Mechanical Engineering,Machine Design - A Comprehensive Guide,2016,J. B. Gupta,2,527.00


'\n'

Ratings Table:


,User_ID,Semester,Ratings
0,1,2,3
1,1,3,2
2,2,1,4
3,2,7,1
4,2,8,1


'\n'

### content based filtering 

In [23]:
import pandas as pd
import mysql.connector
import ipywidgets as widgets
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ✅ Connect to MySQL
def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="educationbr"
    )

# ✅ Fetch books dataset
def get_books_data():
    conn = get_db_connection()
    query = "SELECT * FROM books;"
    books_df = pd.read_sql(query, conn)
    conn.close()
    return books_df

# ✅ Fetch distinct users
def get_users():
    conn = get_db_connection()
    query = "SELECT DISTINCT User_ID FROM ratings;"
    users_df = pd.read_sql(query, conn)
    conn.close()
    return users_df['User_ID'].astype(str).tolist()

# ✅ Fetch distinct branches from books dataset
def get_branches():
    conn = get_db_connection()
    query = "SELECT DISTINCT Branch FROM books;"
    branches_df = pd.read_sql(query, conn)
    conn.close()
    return branches_df['Branch'].dropna().tolist()

# ✅ Content-Based Recommendation Function
def recommend_books(branch, semester):
    books_df = get_books_data()
    
    # Filter books based on branch and semester
    books_df = books_df[(books_df["Branch"] == branch) & (books_df["Semester"] == semester)]
    
    if books_df.empty:
        return pd.DataFrame({"Message": ["No books found for the selected criteria"]})
    
    # Combine features for similarity
    books_df["combined_features"] = books_df["Branch"] + " " + books_df["Book_Title"] + " " + books_df["Author"]

    # ✅ TF-IDF Vectorization
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(books_df["combined_features"])

    # ✅ Compute similarity
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Recommend top books
    book_indices = cosine_sim.sum(axis=0).argsort()[-5:][::-1]  # Top 5 books
    recommended_books = books_df.iloc[book_indices][["Book_Title", "Author", "Branch", "Semester"]]
    
    return recommended_books.reset_index(drop=True)

# ✅ Create Dropdown Widgets
users = get_users()
branches = get_branches()

user_select = widgets.Dropdown(
    options=["New User"] + users,
    description="User:",
    style={'description_width': 'initial'}
)

branch_select = widgets.Dropdown(
    options=branches,
    description="Branch:",
    style={'description_width': 'initial'}
)

semester_select = widgets.Dropdown(
    options=[str(i) for i in range(1, 9)],
    description="Semester:",
    style={'description_width': 'initial'}
)

recommend_button = widgets.Button(description="Get Recommendations", button_style='success')

output = widgets.Output()

# ✅ Function to Display Recommendations
def get_recommendations(b):
    with output:
        output.clear_output()
        branch = branch_select.value
        semester = int(semester_select.value)
        
        recommended_books = recommend_books(branch, semester)
        
        display(recommended_books)

recommend_button.on_click(get_recommendations)

# ✅ Display UI in Jupyter Notebook
display(user_select, branch_select, semester_select, recommend_button, output)


Dropdown(description='User:', options=('New User', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '1…

Dropdown(description='Branch:', options=('Chemical Engineering', 'Computer Science Engineering', 'Electronics …

Dropdown(description='Semester:', options=('1', '2', '3', '4', '5', '6', '7', '8'), style=DescriptionStyle(des…

Button(button_style='success', description='Get Recommendations', style=ButtonStyle())

Output()

### collabrative based filtering 

In [28]:
import mysql.connector
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# ✅ Function to connect and fetch data from MySQL
def fetch_data(query, params=None):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="educationbr"
    )
    cursor = conn.cursor(dictionary=True)
    cursor.execute(query, params or ())
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return pd.DataFrame(data)

# ✅ Fetch User IDs and Branches
user_df = fetch_data("SELECT DISTINCT User_ID FROM Ratings")
branches_df = fetch_data("SELECT DISTINCT Branch FROM Books")

user_ids = sorted(user_df["User_ID"].astype(str).tolist()) + ["New User"]
branches = sorted(branches_df["Branch"].astype(str).tolist())

# ✅ Create UI Elements
user_select = widgets.Dropdown(options=user_ids, description="User ID:")
new_user_input = widgets.Text(placeholder="Enter User ID")
semester_select = widgets.Dropdown(options=[str(i) for i in range(1, 9)], description="Semester:")
new_semester_input = widgets.Dropdown(options=[str(i) for i in range(1, 9)], description="New Semester:")
branch_select = widgets.Dropdown(options=branches, description="Branch:")

recommend_button = widgets.Button(description="Get Recommendations", button_style="primary")
output_area = widgets.Output()

# ✅ Ensure correct layout by making inputs **vertical**
user_layout = widgets.VBox([user_select, new_user_input])
semester_layout = widgets.VBox([semester_select, new_semester_input])
branch_layout = widgets.VBox([branch_select])
button_layout = widgets.VBox([recommend_button])

# ✅ Update UI when user selects "New User"
def update_ui(change):
    if user_select.value == "New User":
        new_user_input.layout.display = "block"
        new_semester_input.layout.display = "block"
        semester_select.layout.display = "none"
    else:
        new_user_input.layout.display = "none"
        new_semester_input.layout.display = "none"
        semester_select.layout.display = "block"
    branch_select.layout.display = "block"

user_select.observe(update_ui, names="value")

# ✅ Fetch recommended books
def get_recommendations():
    selected_user = user_select.value
    selected_branch = branch_select.value
    
    if not selected_branch:
        return "⚠️ Please select a branch."

    if selected_user == "New User":
        selected_semester = new_semester_input.value
    else:
        selected_semester = semester_select.value

    if not selected_semester:
        return "⚠️ Please select a semester."

    selected_semester = int(selected_semester)
    next_semesters = [selected_semester + i for i in range(1, 3) if selected_semester + i <= 8]

    if not next_semesters:
        return "✅ No further semester recommendations available."

    query = f"""
        SELECT Book_Title, Author, Year_of_Publication, Price, Semester
        FROM Books
        WHERE Semester IN ({', '.join(map(str, next_semesters))})
        AND Branch = %s
        ORDER BY Price ASC
        LIMIT 5
    """

    books_df = fetch_data(query, (selected_branch,))

    if books_df.empty:
        return "🚫 No books found for the selected criteria."

    return books_df

# ✅ Button Click Event
def show_recommendations(b):
    with output_area:
        clear_output(wait=True)
        result = get_recommendations()
        display(result if isinstance(result, pd.DataFrame) else widgets.HTML(f"<b>{result}</b>"))

recommend_button.on_click(show_recommendations)

# ✅ Display UI in Jupyter Notebook (Properly aligned)
display(
    widgets.VBox([
        widgets.Label("📚 Book Recommendation System", style={"font-weight": "bold", "font-size": "18px"}),
        user_layout,  # User selection on a separate line
        semester_layout,  # Semester selection on a separate line
        branch_layout,  # Branch selection on a separate line
        button_layout,  # Button below branch selection
        output_area  # Displays results below everything
    ])
)


### HYBRID FILTERING

In [35]:
import mysql.connector
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# ✅ Function to connect and fetch data from MySQL
def fetch_data(query, params=None):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="educationbr"
    )
    cursor = conn.cursor(dictionary=True)
    cursor.execute(query, params or ())
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    return pd.DataFrame(data)

# ✅ Fetch User IDs and Branches
user_df = fetch_data("SELECT DISTINCT User_ID FROM Ratings")
branches_df = fetch_data("SELECT DISTINCT Branch FROM Books")

user_ids = sorted(user_df["User_ID"].astype(str).tolist()) + ["New User"]
branches = sorted(branches_df["Branch"].astype(str).tolist())

# ✅ Create UI Elements
user_select = widgets.Dropdown(options=user_ids, description="User ID:")
new_user_input = widgets.Text(placeholder="Enter New User ID")
semester_select = widgets.Dropdown(options=[str(i) for i in range(1, 9)], description="Semester:")
new_semester_input = widgets.Dropdown(options=[str(i) for i in range(1, 9)], description="New Semester:")
branch_select = widgets.Dropdown(options=branches, description="Branch:")

recommend_button = widgets.Button(description="Get Recommendations", button_style="primary")
output_area = widgets.Output()

# ✅ Hide new user fields initially
new_user_input.layout.display = "none"
new_semester_input.layout.display = "none"

# ✅ Update UI when user selects "New User"
def update_ui(change):
    if user_select.value == "New User":
        new_user_input.layout.display = "block"
        new_semester_input.layout.display = "block"
        semester_select.layout.display = "none"
    else:
        new_user_input.layout.display = "none"
        new_semester_input.layout.display = "none"
        semester_select.layout.display = "block"

user_select.observe(update_ui, names="value")

# ✅ Fetch recommended books
def get_recommendations():
    selected_user = user_select.value
    selected_branch = branch_select.value

    if not selected_branch:
        return "⚠️ Please select a branch."

    selected_semester = int(new_semester_input.value) if selected_user == "New User" else int(semester_select.value)

    next_semesters = [selected_semester + i for i in range(0, 3) if selected_semester + i <= 8]

    if not next_semesters:
        return "✅ No further semester recommendations available."

    query = f"""
        SELECT DISTINCT Book_Title, Author, Semester, Branch
        FROM Books
        WHERE Semester IN ({', '.join(map(str, next_semesters))})
        AND Branch = %s
    """

    books_df = fetch_data(query, (selected_branch,))

    books_df = books_df.drop_duplicates(subset=['Book_Title'])

    return books_df if not books_df.empty else "🚫 No books found for the selected criteria."

# ✅ Button Click Event
def show_recommendations(b):
    with output_area:
        clear_output(wait=True)
        result = get_recommendations()
        display(result if isinstance(result, pd.DataFrame) else widgets.HTML(f"<b>{result}</b>"))

recommend_button.on_click(show_recommendations)

# ✅ Display UI in Jupyter Notebook (Proper Layout)
display(
    widgets.VBox([
        widgets.Label("📚 Book Recommendation System", style={"font-weight": "bold", "font-size": "18px"}),
        user_select,
        new_user_input,
        semester_select,
        new_semester_input,
        branch_select,
        recommend_button,
        output_area  # Results appear here
    ])
)


### SAVING THE MODELS

In [38]:
import pickle
import os
import pandas as pd
import mysql.connector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# :white_check_mark: Ensure the 'models' directory exists
os.makedirs("./models", exist_ok=True)
# :white_check_mark: Connect to MySQL Database using mysql.connector
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="educationbr"
)
# :white_check_mark: Load book data from MySQL
query = "SELECT Branch, Book_Title, Author FROM Books"
books_df = pd.read_sql(query, con=db_connection)
# :white_check_mark: Close database connection
db_connection.close()
# :white_check_mark: Prepare text features for recommendation
books_df["combined_features"] = books_df["Branch"] + " " + books_df["Book_Title"] + " " + books_df["Author"]
# :white_check_mark: Train TF-IDF Vectorizer on books data
tfidf_vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf_vectorizer.fit_transform(books_df["combined_features"])
# :white_check_mark: Compute Cosine Similarity Matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
# :white_check_mark: Save the models in the 'models' folder
pickle.dump(tfidf_vectorizer, open("./models/tfidf_vectorizer.sav", "wb"))
pickle.dump(cosine_sim_matrix, open("./models/cosine_similarity_matrix.sav", "wb"))
print(":white_check_mark: Models saved successfully in the 'models' folder.")

:white_check_mark: Models saved successfully in the 'models' folder.
